In [ ]:
import os
import requests
import pandas as pd
import boto3
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json

load_dotenv()

API_KEY = os.getenv("AEMET_API_KEY")

BUCKET = "aemet-historico-cadiz"
PREFIX = "observaciones/"

s3 = boto3.client("s3")


def descargar_dia(fecha):
    fecha_str = fecha.strftime("%Y-%m-%dT00:00:00UTC")

    url = f"https://opendata.aemet.es/opendata/api/observacion/convencional/datos/fecha/{fecha_str}/fecha/{fecha_str}/estacion/5402"  

    headers = {"api_key": API_KEY}

    r = requests.get(url, headers=headers).json()

    datos_url = r["datos"]

    datos = requests.get(datos_url).json()

    return datos


def pipeline(inicio, fin):

    fecha = inicio
    todos = []

    while fecha <= fin:
        print("Descargando:", fecha.date())

        try:
            datos = descargar_dia(fecha)
            todos.extend(datos)
        except:
            print("Error día", fecha)

        fecha += timedelta(days=1)

    df = pd.DataFrame(todos)


    df["fecha"] = pd.to_datetime(df["fecha"])

  
    file_name = "cadiz_historico.parquet"
    df.to_parquet(file_name, index=False)


    s3.upload_file(file_name, BUCKET, PREFIX + file_name)

    print(" Subido a S3 correctamente")



inicio = datetime(2016, 1, 1)
fin = datetime(2025, 12, 31)

pipeline(inicio, fin)


In [ ]:
import os
import json
import requests
import boto3
from datetime import datetime, timedelta


API_KEY = os.environ.get("AEMET_API_KEY")
BUCKET = "cadiz-observaciones"
s3 = boto3.client("s3")

def lambda_handler(event, context):
    # Últimas 24 horas
    hoy = datetime.utcnow()
    ayer = hoy - timedelta(days=1)
    
    # Formato AEMET
    fecha_inicio = ayer.strftime("%Y-%m-%dT00:00:00UTC")
    fecha_fin   = hoy.strftime("%Y-%m-%dT00:00:00UTC")

    # URL
    url = f"https://opendata.aemet.es/opendata/api/observacion/convencional/datos/fecha/{fecha_inicio}/fecha/{fecha_fin}/estacion/5402"
    headers = {"api_key": API_KEY}

    try:
        r = requests.get(url, headers=headers).json()
        datos_url = r["datos"]
        datos = requests.get(datos_url).json()
        
        
        file_name = f"observaciones_{ayer.strftime('%Y%m%d')}.json"
        with open("/tmp/" + file_name, "w", encoding="utf-8") as f:
            json.dump(datos, f, ensure_ascii=False, indent=2)
        
        # Subir a S3
        s3.upload_file(f"/tmp/{file_name}", BUCKET, file_name)
        print(f" Subido a S3: {file_name}")

        return {"status": "success", "file": file_name}

    except Exception as e:
        print(" Error:", e)
        return {"status": "error", "message": str(e)}


In [ ]:
import json
import boto3
import pandas as pd
from io import BytesIO

s3 = boto3.client("s3")

def lambda_handler(event, context):
    # Obtener info del archivo subido
    bucket = event['Records'][0]['s3']['bucket']['name']
    key    = event['Records'][0]['s3']['object']['key']
    
    print(f" Nuevo archivo: s3://{bucket}/{key}")
    
    try:
        # Descargar objeto S3
        obj = s3.get_object(Bucket=bucket, Key=key)
        data = json.loads(obj['Body'].read().decode('utf-8'))
        
        
        df = pd.DataFrame(data)
        
       
        numeric_cols = ["altitud","tmed","prec","tmin","horatmin","tmax","horatmax",
                        "dir","velmedia","racha","horaracha","hrMedia","hrMax","horaHrMax","hrMin","horaHrMin"]
        
        for c in numeric_cols:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors='coerce')
        
        # Guardar parquet en S3 en otra carpeta
        output_key = key.replace(".json", ".parquet").replace("observaciones/", "parquet/")
        buffer = BytesIO()
        df.to_parquet(buffer, index=False, compression="snappy")
        buffer.seek(0)
        
        s3.put_object(Bucket=bucket, Key=output_key, Body=buffer.getvalue())
        
        print(f" Archivo procesado y subido: s3://{bucket}/{output_key}")
        
        return {"status": "success", "key": output_key}
    
    except Exception as e:
        print(" Error:", e)
        return {"status": "error", "message": str(e)}
